In [ ]:
# Purpose: Gather eligible .jpg paths, sample without replacement, copy preserving structure; final summary print only
from pathlib import Path
import shutil, random

input_root = Path(INPUT_ROOT)
output_root = Path(OUTPUT_ROOT)

# Collect eligible JPGs from size folders 20-120 only
eligible = []
for jpg in input_root.rglob('*.jpg'):
    rel = jpg.relative_to(input_root)
    parts = rel.parts
    # Expected layout: frame_xxxxx/<size>/image.jpg; be robust if deeper
    size_folder = parts[1] if len(parts) >= 3 else (parts[-2] if len(parts) >= 2 else None)
    if size_folder in ALLOWED_SIZES:
        eligible.append(jpg)

# Deduplicate while preserving order (safety; filenames are unique by path)
eligible = list(dict.fromkeys(eligible))

if not eligible:
    print('0 copied: no eligible images found')  # concise final print
else:
    rnd = random.Random(SEED)
    k = min(TARGET_COUNT, len(eligible))
    selected = rnd.sample(eligible, k)  # no duplicates by construction
    for src in selected:
        rel = src.relative_to(input_root)
        dst = output_root / rel
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst)
    print(f'{len(selected)} copied to {OUTPUT_ROOT}')  # concise final print